In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from datetime import datetime

In [3]:
from xrsig import get_emg
from sglxarray import load_trigger
import ecephys_analyses as ea

In [4]:
EMG_OPTS = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')

In [6]:
def get_condition_emg(subject, experiment, condition, probe):
    emg_chans = ea.get_channels(subject, experiment, probe, "emg_from_lfp")
    
    bin_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="lf.bin")
    emg_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="emg.nc")

    for bin_path, emg_path in zip(bin_paths, emg_paths):
        sig = load_trigger(bin_path, emg_chans)
        emg = get_emg(sig, **EMG_OPTS)
        
        # Create parent directories if they do not already exist.
        Path(emg_path).parent.mkdir(parents=True, exist_ok=True)  
        emg.to_netcdf(emg_path)
        emg.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [ ]:
get_condition_emg("Luigi", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
15:36:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
15:45:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
15:54:57: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
16:04:31: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
16:13:50: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000070
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.0096305, LFP nchans = 3




Done!
16:25:19: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX9-Luigi/6-20-2021/6-20-2021_g0/6-20-2021_g0_imec0/6-20-2021_g0_t5.imec0.lf.bin
nChan: 385, nFileSamp: 18000071
